In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#import NDC deliveries csv
deliveries = pd.read_csv('../data/Nash Diaper Stastistics - 2020 Deliveries.csv')
print(deliveries.shape)
deliveries.head()

(766, 16)


,Partners,Date,NB,1,2,3,4,5,6,Pullups,Total,Running Total,Ever to Date,Year to Date,Unnamed: 14,Unnamed: 15
0,NICE Highlands Apartments,1/4/2020,500.0,400.0,450.0,600.0,600.0,600.0,600.0,NaN,"3,750","3,750","2,321,750",NaN,NaN,NaN
1,Martha O'Bryan,1/7/2020,NaN,NaN,NaN,NaN,600.0,600.0,600.0,NaN,"1,800","5,550","2,323,550",NaN,NaN,NaN
2,Madison Hispanic 7th Day,1/7/2020,NaN,NaN,450.0,450.0,500.0,750.0,750.0,NaN,"2,900","8,450","2,326,450",NaN,"2,900",NaN
3,Legacy Mission Village,1/7/2020,1000.0,1000.0,1600.0,2375.0,4500.0,4800.0,4800.0,NaN,"20,075","28,525","2,346,525",NaN,NaN,NaN
4,Tusculum Elementary,1/7/2020,150.0,175.0,150.0,150.0,150.0,150.0,150.0,NaN,"1,075","29,600","2,347,600",NaN,NaN,NaN


In [3]:
#import NDC deliveries csv
partners = pd.read_csv('../data/Nash Diaper Stastistics - 2020 Partner Agencies.csv')
print(partners.shape)
partners.head()

(119, 2)


,Partner,Address
0,Alex Green Elementary,"3921 Lloyd Rd, Whites Creek, TN 37189"
1,Antioch High School,"1900 Hobson Pike, Antioch, TN 37013"
2,Catholic Charities,"2806 McGavock Pk, Nashville, TN 37214"
3,Cole Elementary,"5060 Colemont Dr, Antioch, TN 37013"
4,Connectus Priest Lake,"2637 Murfreesboro Pike, Nashville, TN 37217"


In [4]:
#merge the two files to get addresses for partners
ndc_deliveries = pd.merge(deliveries, partners, left_on = 'Partners', right_on = 'Partner', how = 'left')
ndc_deliveries.head(50)

,Partners,Date,NB,1,2,3,4,5,6,Pullups,Total,Running Total,Ever to Date,Year to Date,Unnamed: 14,Unnamed: 15,Partner,Address
0,NICE Highlands Apartments,1/4/2020,500.0,400.0,450.0,600.0,600.0,600.0,600.0,NaN,"3,750","3,750","2,321,750",NaN,NaN,NaN,NICE Highlands Apartments,"4646 Nolensville Pike, Nashville, TN 37211"
1,Martha O'Bryan,1/7/2020,NaN,NaN,NaN,NaN,600.0,600.0,600.0,NaN,"1,800","5,550","2,323,550",NaN,NaN,NaN,Martha O'Bryan,"711 S 7th St, Nashville, TN 37206"
2,Madison Hispanic 7th Day,1/7/2020,NaN,NaN,450.0,450.0,500.0,750.0,750.0,NaN,"2,900","8,450","2,326,450",NaN,"2,900",NaN,Madison Hispanic 7th Day,"1605 Neelys Bend Rd, Nashville, TN 37115"
3,Legacy Mission Village,1/7/2020,1000.0,1000.0,1600.0,2375.0,4500.0,4800.0,4800.0,NaN,"20,075","28,525","2,346,525",NaN,NaN,NaN,Legacy Mission Village,"5123 Nolensville Pike, Nashville, TN 37211"
4,Tusculum Elementary,1/7/2020,150.0,175.0,150.0,150.0,150.0,150.0,150.0,NaN,"1,075","29,600","2,347,600",NaN,NaN,NaN,Tusculum Elementary,"440 McMurray Dr, Nashville, TN 37211"
5,Cole Elementary,1/7/2020,NaN,NaN,NaN,375.0,300.0,300.0,300.0,NaN,"1,275","30,875","2,348,875",NaN,NaN,NaN,Cole Elementary,"5060 Colemont Dr, Antioch, TN 37013"
6,Glenview Elementary,1/7/2020,NaN,400.0,NaN,NaN,400.0,450.0,450.0,NaN,"1,700","32,575","2,350,575",NaN,"1,700",NaN,Glenview Elementary,"1020 Patricia Dr, Nashville, TN 37217"
7,Glencliff High School,1/9/2020,NaN,NaN,NaN,NaN,400.0,400.0,450.0,NaN,"1,250","33,825","2,351,825",NaN,NaN,NaN,Glencliff High School,"160 Antioch Pike, Nashville, TN 37211"
8,Nurses for Newborns,1/10/2020,250.0,NaN,NaN,300.0,NaN,300.0,NaN,NaN,850,"34,675","2,352,675",NaN,850,NaN,Nurses for Newborns,"50 Vantage Way # 101, Nashville, TN 37228"
9,NeedLink Nashville,1/13/2020,NaN,225.0,NaN,300.0,300.0,300.0,450.0,NaN,"1,575","36,250","2,354,250",NaN,"1,575",NaN,NeedLink Nashville,"1600 56th Ave N, Nashville, TN 37209"


In [6]:
#check for null addresses
ndc_deliveries[ndc_deliveries['Address'].isnull()]
#don't have addresses for below entries

,Partners,Date,NB,1,2,3,4,5,6,Pullups,Total,Running Total,Ever to Date,Year to Date,Unnamed: 14,Unnamed: 15,Partner,Address
67,Team Music Is Love One Gen Away,3/6/2020,900.0,1050.0,1000.0,1600.0,800.0,825.0,750.0,NaN,"6,925","69,559","2,532,824",NaN,NaN,NaN,Team Music Is Love One Gen Away,NaN
149,Salvation Army,4/6/2020,0.0,0.0,250.0,250.0,150.0,250.0,200.0,NaN,"1,100","38,800","2,700,105",NaN,NaN,NaN,Salvation Army,NaN
295,Andrea Hollis,5/15/2020,0.0,0.0,0.0,0.0,46.0,0.0,0.0,NaN,46,"127,978","3,010,758",NaN,NaN,NaN,Andrea Hollis,NaN
393,Andrea Hollis,6/15/2020,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,100,"112,223","3,230,719",NaN,NaN,NaN,Andrea Hollis,NaN
490,Andrea Hollis,7/17/2020,NaN,NaN,NaN,NaN,150.0,NaN,NaN,NaN,150,"111,735","3,457,726","1,141,076","32,945",NaN,Andrea Hollis,NaN
581,Andrea Hollis,9/2/2020,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,50,"17,325","3,726,846",NaN,NaN,NaN,Andrea Hollis,NaN
